## Connect to database

In [1]:
import os
import psycopg2
import pandas as pd
import platform

host = "localhost" if platform.system() == "Windows" else "129.108.49.49"

conn = psycopg2.connect(dbname="deeplabcut_db", user="postgres", password="1234", host=host, port="5432")

cursor = conn.cursor()


## Code to Add All Subdirs of "Python_scripts" to sys.path

In [2]:
import sys
from pathlib import Path

# Add Python_scripts to sys.path (parent of Extract_db_columns)
scripts_dir = Path().resolve().parents[0] / "Python_scripts"
sys.path.append(str(scripts_dir))

# Add the *project root* to sys.path (i.e., the parent of Python_scripts)
project_root = Path().resolve().parents[0]  # Adjust if notebook is nested deeper
sys.path.append(str(project_root))


In [6]:
# # cursor.execute("DELETE FROM dlc_table WHERE id = %s;", (532,))

# # conn.commit()

# cursor.execute("""
#     UPDATE dlc_table
#     SET video_name = %s
#     WHERE id = %s;
# """, ('ToyLightInhibitory_1_22_25_S4P_P.mp4', 533))

# conn.commit()


## Insert video file names (Run this from the computer where the videos are located)

In [9]:
from pathlib import Path

project_root = Path.home() / "Downloads" / "DeepLabCutProjects" / "data"
base_folder = project_root.joinpath("WhiteAnimals", "10X", "ToyLight", "SplitVideos")

print(f"Looking in: {base_folder}, Exists? {base_folder.exists()}")

# Collect video files
video_files = list(base_folder.rglob("*.mp4")) + list(base_folder.rglob("*.avi"))

# Create relative path or name
video_records = [(vf.name,) for vf in video_files]

# Get existing entries
cursor.execute("SELECT video_name FROM dlc_table;")
existing = set(row[0] for row in cursor.fetchall())

# Filter only new
new_records = [vr for vr in video_records if vr[0] not in existing]

# Insert
if new_records:
    insert_query = "INSERT INTO dlc_table (video_name) VALUES (%s);"
    cursor.executemany(insert_query, new_records)
    conn.commit()
    print(f"Inserted {len(new_records)} new videos.")
else:
    print("No new videos to insert.")


Looking in: /Users/atanugiri/Downloads/DeepLabCutProjects/data/WhiteAnimals/10X/ToyLight/SplitVideos, Exists? True
Inserted 19 new videos.


## Insert task, date_str, name, health, id_ in dlc_table

In [10]:
import importlib
import Extract_db_columns.parse_video_name
importlib.reload(Extract_db_columns.parse_video_name)
from Extract_db_columns.parse_video_name import parse_video_name

cursor.execute("SELECT id, video_name FROM dlc_table WHERE video_name IS NOT NULL AND task IS NULL;")
rows = cursor.fetchall()

updates = []

for row in rows:
    id_, video_name = row
    task, date_str, name, health = parse_video_name(video_name)
    updates.append((task, date_str, name, health, id_))

# Update in batch
cursor.executemany("""
UPDATE dlc_table
SET task = %s, date = %s, name = %s, health = %s
WHERE id = %s;
""", updates)

conn.commit()


## Insert num_frames, frame_rate, video_width, video_height

In [16]:
import importlib
import Extract_db_columns.update_video_info

importlib.reload(Extract_db_columns.update_video_info)
from Extract_db_columns.update_video_info import update_video_info_in_db

# Call the function with subdirectory
base_video_dir = Path.home() / "Downloads" / "DeepLabCutProjects" / "data"
video_subdir = "WhiteAnimals/10X/ToyLight"

update_video_info_in_db(conn, base_video_dir, video_subdir)


ID 994: ToyLight_8_6_25_S1P_Kale.mp4 — 30016 frames, 25.00 fps, 360x288
ID 995: ToyLight_8_7_25_S3Y_Cherry.mp4 — 30016 frames, 25.00 fps, 360x288
ID 996: ToyLight_8_7_25_S2Y_Apple.mp4 — 30017 frames, 25.00 fps, 360x288
ID 997: ToyLight_8_7_25_S1Y_Broccoli.mp4 — 30016 frames, 25.00 fps, 360x288
ID 998: ToyLight_8_7_25_S1Y_Celery.mp4 — 30016 frames, 25.00 fps, 360x288
ID 999: ToyLight_8_6_25_S2P_Kiwi.mp4 — 30019 frames, 25.00 fps, 360x288
ID 1000: ToyLight_8_6_25_S1P_Cauliflower.mp4 — 30016 frames, 25.00 fps, 360x288
ID 1001: ToyLight_8_7_25_S2Y_Melon.mp4 — 30017 frames, 25.00 fps, 360x288
ID 1002: ToyLight_8_6_25_S3P_Berry.mp4 — 30015 frames, 25.00 fps, 360x288
ID 1003: ToyLight_8_6_25_S3P_Tomato.mp4 — 30015 frames, 25.00 fps, 360x288
ID 1004: ToyLight_8_6_25_S2P_Fig.mp4 — 30019 frames, 25.00 fps, 360x288
ID 1005: ToyLight_8_7_25_S3Y_Mango.mp4 — 30016 frames, 25.00 fps, 360x288
ID 1006: ToyLight_8_6_25_S3P_Mushroom.mp4 — 30015 frames, 25.00 fps, 360x288
ID 1007: ToyLight_8_7_25_S1Y_Spin

## Insert trial_length

In [17]:
import importlib
import Extract_db_columns.insert_trial_length

importlib.reload(Extract_db_columns.insert_trial_length)
from Extract_db_columns.insert_trial_length import insert_trial_length

query = "SELECT id FROM dlc_table WHERE trial_length IS NULL ORDER BY id"

df = pd.read_sql_query(query, conn)
id_list = df['id'].to_list()

insert_trial_length(id_list, conn)


/var/folders/_n/c65sg0c951g6vgwcpzsdc2tc0000gn/T/ipykernel_72894/1939537613.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)
Updating trial_length: 100%|████████████████████████████████████████████████████████████████████| 111/111 [00:02<00:00, 53.36it/s]

[INFO] Updated trial_length for 111 rows.


## Insert genotype

In [18]:
cursor = conn.cursor()
cursor.execute("UPDATE dlc_table SET genotype = 'white' WHERE genotype is NULL")
conn.commit()


## Insert maze_number

In [29]:
from Extract_db_columns.extract_maze_number import (
    load_mother_videos, build_prefix_to_animal_map, get_maze_number, update_maze_numbers_in_db
)

raw_video_dirs = [
    "/Users/atanugiri/Downloads/DeepLabCutProjects/data/WhiteAnimals/10X/LightOnly/RawVideos"
]

mother_videos = load_mother_videos(raw_video_dirs)
prefix_to_animals = build_prefix_to_animal_map(mother_videos)

update_maze_numbers_in_db(conn, "dlc_table", prefix_to_animals)


/Users/atanugiri/Downloads/DeepLabCutProjects/GhrelinBehaviorQuantification/Python_scripts/Extract_db_columns/extract_maze_number.py:118: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT id, video_name, name FROM {dlc_table_name}", conn)


[✓] LightOnly_7_25_25_S1Y_Spinach.mp4 → LightOnly_7_25_25_S1Y_Celery_Broccoli_Spinach_None_Trial_1.mp4 → maze 3
[✓] LightOnly_7_25_25_S3Y_Mango.mp4 → LightOnly_7_25_25_S3Y_Mango_Cherry_Potato_Squash_Trial_1.mp4 → maze 1
[✓] LightOnly_7_25_25_S3Y_Squash.mp4 → LightOnly_7_25_25_S3Y_Mango_Cherry_Potato_Squash_Trial_1.mp4 → maze 4
[✓] LightOnly_7_28_25_S3P_Berry.mp4 → LightOnly_7_28_25_S3P_Peach_Berry_Mushroom_Tomato_Trial_1.mp4 → maze 2
[✓] LightOnly_7_28_25_S3P_Peach.mp4 → LightOnly_7_28_25_S3P_Peach_Berry_Mushroom_Tomato_Trial_1.mp4 → maze 1
[✓] LightOnly_7_25_25_S3Y_Potato.mp4 → LightOnly_7_25_25_S3Y_Mango_Cherry_Potato_Squash_Trial_1.mp4 → maze 3
[✓] LightOnly_7_28_25_S2P_Kiwi.mp4 → LightOnly_7_28_25_S2P_Fig_Kiwi_Guava_None_Trial_1.mp4 → maze 2
[✓] LightOnly_7_25_25_S2Y_Melon.mp4 → LightOnly_7_25_25_S2Y_Melon_Apple_None_None_Trial_1.mp4 → maze 1
[✓] LightOnly_7_25_25_S2Y_Apple.mp4 → LightOnly_7_25_25_S2Y_Melon_Apple_None_None_Trial_1.mp4 → maze 2
[✓] LightOnly_7_28_25_S2P_Fig.mp4 → Li

## Insert csv_file_path

In [28]:
import importlib
from tqdm import tqdm

import Python_scripts.Extract_db_columns.find_csv_for_video as find_mod
importlib.reload(find_mod)
from Python_scripts.Extract_db_columns.find_csv_for_video import find_csv_for_video

# Query for 'FoodOnly%' tasks
query = """
SELECT id, video_name
FROM dlc_table
WHERE task='LightOnly' AND genotype='white' AND frame_rate IS NOT NULL AND dose_mult=10
ORDER BY id;
"""
df = pd.read_sql_query(query, conn)
# df.to_csv("dlc_table_FoodOnly.csv", index=False)

# Directory to search
dir_to_search = (Path.home() / "Downloads" / "DeepLabCutProjects" / "data" / "WhiteAnimals" / 
                  "10X" / "LightOnly" / "DlcDataPytorchFiltered")

# Loop and update
for _, row in tqdm(df.iterrows(), total=len(df), desc="Inserting CSV path for LightOnly"):
    video_id = row["id"]
    video_name = row["video_name"]
    csv_path = find_csv_for_video(video_name, dir_to_search)
    print(csv_path)

    if csv_path:
        cursor.execute(
            "UPDATE dlc_table SET csv_file_path = %s WHERE id = %s;",
            (csv_path, video_id)
        )
    else:
        print(f"❌ No CSV found for {video_name}")

conn.commit()


/var/folders/_n/c65sg0c951g6vgwcpzsdc2tc0000gn/T/ipykernel_73017/3461838021.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)
Inserting CSV path for LightOnly: 100%|██████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 144.47it/s]

/Users/atanugiri/Downloads/DeepLabCutProjects/data/WhiteAnimals/10X/LightOnly/DlcDataPytorchFiltered/LightOnly_7_25_25_S1Y_SpinachDLC_Resnet50_DLC-WhiteAnimalsJul23shuffle1_snapshot_060_filtered.csv
/Users/atanugiri/Downloads/DeepLabCutProjects/data/WhiteAnimals/10X/LightOnly/DlcDataPytorchFiltered/LightOnly_7_25_25_S3Y_PotatoDLC_Resnet50_DLC-WhiteAnimalsJul23shuffle1_snapshot_060_filtered.csv
/Users/atanugiri/Downloads/DeepLabCutProjects/data/WhiteAnimals/10X/LightOnly/DlcDataPytorchFiltered/LightOnly_7_28_25_S2P_KiwiDLC_Resnet50_DLC-WhiteAnimalsJul23shuffle1_snapshot_060_filtered.csv
/Users/atanugiri/Downloads/DeepLabCutProjects/data/WhiteAnimals/10X/LightOnly/DlcDataPytorchFiltered/LightOnly_7_25_25_S2Y_MelonDLC_Resnet50_DLC-WhiteAnimalsJul23shuffle1_snapshot_060_filtered.csv
/Users/atanugiri/Downloads/DeepLabCutProjects/data/WhiteAnimals/10X/LightOnly/DlcDataPytorchFiltered/LightOnly_7_25_25_S3Y_SquashDLC_Resnet50_DLC-WhiteAnimalsJul23shuffle1_snapshot_060_filtered.csv
/Users/atanu

## Add modulation column

In [3]:
SQLS = [
    "ALTER TABLE dlc_table ADD COLUMN IF NOT EXISTS modulation TEXT;",
    """
    UPDATE dlc_table
    SET modulation = 'Inhibitory',
        task = TRIM(REPLACE(task, 'Inhibitory', ''))
    WHERE task ILIKE '%Inhibitory';
    """,
    """
    UPDATE dlc_table
    SET modulation = 'Excitatory',
        task = TRIM(REPLACE(task, 'Excitatory', ''))
    WHERE task ILIKE '%Excitatory';
    """,
    "UPDATE dlc_table SET modulation = 'NA' WHERE modulation IS NULL;",
]

try:
    for stmt in SQLS:
        cursor.execute(stmt)
    conn.commit()
    print("Done.")


Done.


## Add dose_mult

In [6]:
SQL = "UPDATE dlc_table SET dose_mult=10 WHERE dose_mult IS NULL"
cursor.execute(SQL)
conn.commit()


## Add maze center

In [3]:
import importlib
import Python_scripts.Extract_db_columns.center_assign as center_assign_mod
importlib.reload(center_assign_mod)
from Python_scripts.Extract_db_columns.center_assign import (
    find_center_for_task, update_center_for_task
)

n = update_center_for_task(conn, table="dlc_table", force=False)
print("updated rows:", n)


updated rows: 564


### Completely Clear Table First (CAREFUL)

In [ ]:
cursor.execute("DELETE FROM dlc_files")
conn.commit()
print("All entries deleted from dlc_files.")
# conn.rollback()